In [2]:
!pip install datasets
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 47.6 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Load the models

In [1]:
from transformers import AutoProcessor, AutoModelForTextToSpectrogram, SpeechT5HifiGan

# Load English model
processor = AutoProcessor.from_pretrained("Tejasva-Maurya/Hindi_SpeechT5_finetuned")
model = AutoModelForTextToSpectrogram.from_pretrained("Tejasva-Maurya/Hindi_SpeechT5_finetuned")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

## Creating Embeddings

In [6]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier
from datasets import load_dataset, Audio

dataset = load_dataset("mozilla-foundation/common_voice_17_0", "hi", split="validated", trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)
def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings
def prepare_dataset(example):
    audio = example["audio"]
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])
    return example


# Calculate the number of rows for a part of the dataset
part = len(dataset) //800

# Select the part of the dataset
dataset = dataset.select(range(part))

# Prepare the dataset
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)
example = dataset[10]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

## Text Preprocessing

In [8]:
import re
def text_preprocessing(text):
     replacements = [
       # Vowels and vowel matras
       ("अ", "a"),
       ("आ", "aa"),
       ("इ", "i"),
       ("ई", "ee"),
       ("उ", "u"),
       ("ऊ", "oo"),
       ("ऋ", "ri"),
       ("ए", "e"),
       ("ऐ", "ai"),
       ("ऑ", "o"),  # More accurate than 'au' for ऑ
       ("ओ", "o"),
       ("औ", "au"),
       # Consonants
       ("क", "k"),
       ("ख", "kh"),
       ("ग", "g"),
       ("घ", "gh"),
       ("ङ", "ng"),  # nasal sound
       ("च", "ch"),
       ("छ", "chh"),
       ("ज", "j"),
       ("झ", "jh"),
       ("ञ", "ny"),  # 'ny' closer to the actual sound
       ("ट", "t"),
       ("ठ", "th"),
       ("ड", "d"),
       ("ढ", "dh"),
       ("ण", "n"),  # Slight improvement for easier pronunciation
       ("त", "t"),
       ("थ", "th"),
       ("द", "d"),
       ("ध", "dh"),
       ("न", "n"),
       ("प", "p"),
       ("फ", "ph"),
       ("ब", "b"),
       ("भ", "bh"),
       ("म", "m"),
       ("य", "y"),
       ("र", "r"),
       ("ल", "l"),
       ("व", "v"),  # 'v' is closer to the Hindi 'व'
       ("श", "sh"),
       ("ष", "sh"),  # Same sound in modern pronunciation
       ("स", "s"),
       ("ह", "h"),
       # Consonant clusters and special consonants
       ("क्ष", "ksh"),
       ("त्र", "tr"),
       ("ज्ञ", "gya"),
       ("श्र", "shra"),
       # Special characters
       ("़", ""),    # Ignore nukta; can vary with regional pronunciation
       ("्", ""),    # Halant - schwa dropping (handled contextually)
       ("ऽ", ""),    # Avagraha - no direct pronunciation, often ignored
       ("ं", "n"),   # Anusvara - nasalization
       ("ः", "h"),   # Visarga - adds an 'h' sound
       ("ँ", "n"),   # Chandrabindu - nasalization
       # Vowel matras (diacritic marks)
       ("ा", "a"),
       ("ि", "i"),
       ("ी", "ee"),
       ("ु", "u"),
       ("ू", "oo"),
       ("े", "e"),
       ("ै", "ai"),
       ("ो", "o"),
       ("ौ", "au"),
       ("ृ", "ri"),  # Vowel-matra equivalent of ऋ
       # Nasalization and other marks
       ("ॅ", "e"),   # Short 'e' sound (very rare)
       ("ॉ", "o"),   # Short 'o' sound (very rare)
       # Loanwords and aspirated consonants
       ("क़", "q"),
       ("ख़", "kh"),
       ("ग़", "gh"),
       ("ज़", "z"),
       ("ड़", "r"),
       ("ढ़", "rh"),
       ("फ़", "f"),
       # Punctuation
       ("।", "."),   # Hindi sentence-ending marker -> period
   ]

     # Remove extra whitespace
     text = ' '.join(text.split())
     for src, dst in replacements:
       text = text.replace(src, dst)
     return text

## Audio generation

In [9]:
text ="आज मौसम बहुत अच्छा है।"
text = text_preprocessing(text)
inputs = processor(text=text, return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [10]:
from IPython.display import Audio
Audio(speech.numpy(), rate = 16000)